In [8]:
import re
import pandas as pd

# create a dataset
data = {'text': ['Im 18F', 'This is my favorite food', '20M'], 'user': ['a', 'b', 'c']}

# convert it to a dataframe
df = pd.DataFrame.from_dict(data)

# check if the text contains a number followed by the letter 'F'
df['text'].str.contains('\dF')

0     True
1    False
2    False
Name: text, dtype: bool

In [19]:
df.loc[df['text'].str.contains(r'\dF'),:]

,text,user
0,Im 18F,a


In [17]:
new_df = df[df['text'].str.contains(r'\dF') | df['text'].str.contains(r'\dM')]

In [18]:
new_df

,text,user
0,Im 18F,a
2,20M,c


In [1]:
import requests
import pandas as pd
import datetime as dt
import time
pd.set_option('display.max_columns', None)

In [2]:
# custom function to query reddit multiple times
# this code is adapted from Hovanes's codes that he shared in class

def reddit_query(subreddit, n_samples):
    base_url = 'https://api.pushshift.io/reddit/submission/search' # base url for api
    list_posts = []
    oldest_post = None
    
    while len(list_posts) < n_samples: # n_samples exceeds the reddit api limit of 1000
        params = {
            "subreddit" : subreddit,
            "size" : 1000,
            "before": oldest_post # a time stamp to set when to start pulling posts
            }  
            
        res = requests.get(base_url,params) # request reddit info
            
        posts = res.json()['data'] # create list from json
        
        if len(posts) == 0:
            oldest_post = None # for the first time running this, before will be None
            list_posts.extend(posts) # add batch of posts to list_posts     
        else:
            time.sleep(3) # wait 3 seconds before querying again
            oldest_post = dt.datetime.fromtimestamp(posts[-1]["created_utc"]) # take the oldest time from the list and pull posts before that time
            list_posts.extend(posts) # add batch of posts to list_posts
    return pd.DataFrame(list_posts) # return pandas dataframe

In [7]:
requests.get('https://api.pushshift.io/reddit/submission/search')

<Response [403]>

In [5]:
rbn = reddit_query("raisedbynarcissists", 10)

KeyError: 'data'